In [0]:
import requests
import pandas as pd
import math

import json
from functools import reduce

# Matplotlib for visualization at the end
import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.sql.functions import col, when, sum, when,lit, udf, to_timestamp, year, month, hour

from pyspark.sql.types import StructType, StructField, StringType, DoubleType

from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

## Grouping the features

In [0]:
wind = ["wind_speed_10m", "wind_speed_120m", "wind_direction_10m", "wind_direction_120m", "wind_gusts_10m"]

cloud_cover = ["cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "visibility"]

# realtive_humidity_2m: is the ratio of the actual water vapor in the air to the maximum the air can hold at that temperature (in %)
# dew_point_2m: temperature at which air becomes saturated with moisture
# vapour_pressure_deficit: difference between the amount of moisture in the air and how much it can hold when saturated.

temperature = ["apparent_temperature", "temperature_2m", "temperature_120m", "relative_humidity_2m", "dew_point_2m", "vapour_pressure_deficit"]

rain_snow = ["precipitation_probability", "precipitation", "rain", "showers", "snowfall", "snow_depth"]

pressure = ["surface_pressure", "pressure_msl"]

water_balance = ["evapotranspiration", "et0_fao_evapotranspiration"]


## Loading the data

In [0]:
url = "https://historical-forecast-api.open-meteo.com/v1/forecast"

dfs={}

# create pandas dataframes with the different types of features
# after transform the pandas data frane into a spark data frame

features = {
    'wind': wind, 
    'cloud_cover': cloud_cover, 
    'temperature': temperature, 
    'rain_snow': rain_snow,
    'pressure': pressure, 
    'water_balance': water_balance
    }

for key, value in features.items():
	params = {
		# Lisbon coordinates
		"latitude": 38.716885,
		"longitude": -9.140233,
		# 1 Year Historical data
		"start_date": "2023-05-01",
		"end_date": "2025-05-01",
		"hourly": value
	}

	response = requests.get(url, params=params)
	data = response.json()
	print(key)
	pdf = pd.DataFrame(data['hourly'])
	dfs[key] = spark.createDataFrame(pdf)

---------------------------------------------------------------------------
JSONDecodeError                           Traceback (most recent call last)
File /databricks/python/lib/python3.9/site-packages/requests/models.py:910, in Response.json(self, **kwargs)
    909 try:
--> 910     return complexjson.loads(self.text, **kwargs)
    911 except JSONDecodeError as e:
    912     # Catch JSON-related errors and raise as requests.JSONDecodeError
    913     # This aliases json.JSONDecodeError and simplejson.JSONDecodeError

File /usr/lib/python3.9/json/__init__.py:346, in loads(s, cls, object_hook, parse_float, parse_int, parse_constant, object_pairs_hook, **kw)
    343 if (cls is None and object_hook is None and
    344         parse_int is None and parse_float is None and
    345         parse_constant is None and object_pairs_hook is None and not kw):
--> 346     return _default_decoder.decode(s)
    347 if cls is None:

File /usr/lib/python3.9/json/decoder.py:337, in JSONDecoder.decod

In [0]:
dfs

Out[26]: {}

In [0]:
# columns on the wind df as an example
dfs['wind'].columns


Out[4]: ['time',
 'wind_speed_10m',
 'wind_speed_120m',
 'wind_direction_10m',
 'wind_direction_120m',
 'wind_gusts_10m']

In [0]:
# list of dataframes extracted from the dictionary
dfs_list = list(dfs.values())

# merge all DataFrames on the 'time' column with an outer join, reduce is used to simplify code
# so that we dont have to do all the joins one by one
df_merged = reduce(lambda left, right: left.join(right, on="time", how="outer"), dfs_list)


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-3007706209336687>:6
      2 dfs_list = list(dfs.values())
      4 # merge all DataFrames on the 'time' column with an outer join, reduce is used to simplify code
      5 # so that we dont have to do all the joins one by one
----> 6 df_merged = reduce(lambda left, right: left.join(right, on="time", how="outer"), dfs_list)

TypeError: reduce() of empty sequence with no initial value

In [0]:
df_merged.write.mode("overwrite").option("header", "true").csv("/dbfs/FileStore/forecast.csv")

## Check for missing values

In [0]:
for group_feat in dfs.keys(): # check missing values
    dfs[group_feat].select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in dfs[group_feat].columns]).show()


+----+--------------+---------------+------------------+-------------------+--------------+
|time|wind_speed_10m|wind_speed_120m|wind_direction_10m|wind_direction_120m|wind_gusts_10m|
+----+--------------+---------------+------------------+-------------------+--------------+
|   0|             0|              0|                 0|                  0|             0|
+----+--------------+---------------+------------------+-------------------+--------------+

+----+-----------+---------------+---------------+----------------+----------+
|time|cloud_cover|cloud_cover_low|cloud_cover_mid|cloud_cover_high|visibility|
+----+-----------+---------------+---------------+----------------+----------+
|   0|          0|              0|              0|               0|         0|
+----+-----------+---------------+---------------+----------------+----------+



In [0]:
df_merged.columns

Out[7]: ['time',
 'wind_speed_10m',
 'wind_speed_120m',
 'wind_direction_10m',
 'wind_direction_120m',
 'wind_gusts_10m',
 'cloud_cover',
 'cloud_cover_low',
 'cloud_cover_mid',
 'cloud_cover_high',
 'visibility']

In [0]:
df_merged=df_merged.drop('precipitation_probability') # drop the only column with missing values (it only has missing values)

## Create columns: months years, hours and seasons

In [0]:
df_merged = df_merged.withColumn("time_ts", to_timestamp("time", "yyyy-MM-dd'T'HH:mm"))


In [0]:
df_merged = df_merged.withColumn("year", year("time_ts")) \
       .withColumn("month", month("time_ts")) \
       .withColumn("hour", hour("time_ts"))

df_merged = df_merged.drop("time_ts")


In [0]:
df_merged = df_merged.withColumn(
    "season",
    when(col("month").isin(12, 1, 2), "Winter")
    .when(col("month").isin(3, 4, 5), "Spring")
    .when(col("month").isin(6, 7, 8), "Summer")
    .when(col("month").isin(9, 10, 11), "Autumn")
)


## Use SQL to do queries

In [0]:
# Create a temporary view to query with sql
df_merged.createOrReplaceTempView("df_merged")

In [0]:
%sql
-- See the table
SELECT *
FROM df_merged

time,wind_speed_10m,wind_speed_120m,wind_direction_10m,wind_direction_120m,wind_gusts_10m,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility
2023-05-01T00:00,6.2,21.4,21,7,18.0,0,0,0,0,24140.0
2023-05-01T01:00,4.5,19.8,14,2,13.0,0,0,0,0,24140.0
2023-05-01T02:00,3.0,18.8,346,355,10.4,0,0,0,0,24140.0
2023-05-01T03:00,3.2,18.8,360,357,6.8,100,0,0,100,24140.0
2023-05-01T04:00,3.0,17.7,346,355,5.8,42,0,0,42,24140.0
2023-05-01T05:00,3.6,19.4,6,360,8.3,96,0,0,96,24140.0
2023-05-01T06:00,3.4,16.7,32,6,9.0,29,0,0,29,24140.0
2023-05-01T07:00,3.3,14.5,41,14,7.2,0,0,0,0,24140.0
2023-05-01T08:00,5.0,11.7,21,18,14.4,0,0,0,0,24140.0
2023-05-01T09:00,7.0,9.2,21,21,19.4,0,0,0,0,24140.0


In [0]:
%sql
-- See the correlations between wind features
SELECT 
corr(wind_speed_10m,
 wind_direction_10m) AS correlation_speed10_direction10,
corr(wind_speed_10m, wind_speed_120m) AS correlation_speed10_speed120,
corr(wind_direction_10m, wind_direction_120m) AS correlation_direction10_direction120,
corr(wind_speed_10m, wind_gusts_10m) AS correlation_speed10_gusts10,
corr(wind_direction_10m, wind_gusts_10m) AS correlation_direction10_gusts10
FROM df_merged;

In [0]:
%sql
-- See the correlations between wind and pressure features
SELECT 
corr(wind_gust_10m, surface_pressure) AS correlation_gusts10_sufpress,
corr(wind_direction_10m, surface_pressure) AS correlation_direction10_sufpress,
corr(wind_gust_10m, pressure_msl) AS correlation_gusts10_mslpress,
corr(wind_direction_10m, pressure_msl) AS correlation_direction10_mslpress,
corr(surface_pressure, pressure_msl) AS correlation_sufpress_mslpress
FROM df_merged;

correlation_speed10_direction10,correlation_speed10_speed120,correlation_direction10_direction120,correlation_speed10_gusts10,correlation_direction10_gusts10
0.22535395172605127,0.8335900366635401,0.8971379164623488,0.9654900879984376,0.2838115525230414


In [0]:
%sql
-- See the correlations between cloud_cover features
SELECT 
corr(cloud_cover_low, cloud_cover_mid) AS correlation_cloud_cover_low_mid,
corr(cloud_cover_low, cloud_cover_high) AS correlation_cloud_cover_low_high,
corr(cloud_cover, cloud_cover_low) AS correlation_cloud_cover_total_low,
corr(cloud_cover, cloud_cover_mid) AS correlation_cloud_cover_total_mid
FROM df_merged;

correlation_cloud_cover_low_mid,correlation_cloud_cover_low_high,correlation_cloud_cover_total_low,correlation_cloud_cover_total_mid
0.33749081873934844,0.17891933322203202,0.656320162621009,0.6139340777978612


In [0]:
%sql
-- See the correlations between cloud_cover and precipitation and visibility features
SELECT 
corr(cloud_cover, visibility) AS correlation_cloud_cover_visibility,
corr(cloud_cover, precipitation) AS correlation_cloud_cover_precipitation,
corr(cloud_cover_high, visibility) AS correlation_cloud_cover_high_visibility,
corr(cloud_cover_high, precipitation) AS correlation_cloud_cover_high_precipitation,
corr(visibility, precipitation) AS correlation_visibility_precipitation
FROM df_merged;

In [0]:
%sql
-- See the correlations between temperature and humidity features
SELECT
corr(temperature_2m, relative_humidity_2m) AS correlation_temp_2m_hum_2m,
corr(temperature_2m, vapour_pressure_deficit) AS correlation_temp_2m_vap_press,
corr(relative_humidity_2m, vapour_pressure_deficit) AS correlation_hum_2m_vap_press
FROM df_merged;

In [0]:
%sql
-- See the top wind speeds
SELECT *
FROM df_merged
ORDER BY wind_speed_10m DESC
LIMIT 20;

time,wind_speed_10m,wind_speed_120m,wind_direction_10m,wind_direction_120m,wind_gusts_10m,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility
2025-03-20T02:00,41.1,82.8,189,191,93.2,75,67,27,0,23820.0
2025-03-20T00:00,39.0,83.6,148,152,82.4,100,80,100,100,12000.0
2023-10-19T11:00,38.2,85.7,234,235,92.5,100,87,100,100,540.0
2025-03-20T01:00,38.2,78.1,178,181,91.4,100,61,100,62,19640.0
2025-03-20T04:00,34.1,70.4,232,234,79.2,94,76,83,0,30460.0
2023-10-19T09:00,33.6,74.6,225,225,78.1,100,76,92,100,9600.0
2023-10-19T10:00,33.6,74.5,228,229,86.8,100,72,100,100,4160.0
2025-03-07T13:00,33.0,69.0,173,175,73.1,100,90,100,100,32100.0
2025-03-20T03:00,32.8,68.2,213,214,88.6,89,71,63,22,12880.0
2025-01-26T13:00,32.7,66.2,207,208,76.7,100,93,86,97,17180.0


In [0]:
%sql
-- See the top cloud cover high
SELECT *
FROM df_merged
ORDER BY cloud_cover_high DESC
LIMIT 20;

time,wind_speed_10m,wind_speed_120m,wind_direction_10m,wind_direction_120m,wind_gusts_10m,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility
2023-05-02T18:00,11.5,22.6,250,253,38.2,100,0,100,100,24140.0
2023-05-02T14:00,17.1,21.1,240,237,36.4,100,0,0,100,24140.0
2023-05-01T03:00,3.2,18.8,360,357,6.8,100,0,0,100,24140.0
2023-05-01T19:00,11.3,21.1,353,351,32.0,100,0,0,100,24140.0
2023-05-02T21:00,6.8,22.1,238,240,14.8,100,0,100,100,24140.0
2023-05-02T00:00,4.3,18.0,66,26,6.5,100,0,0,100,24140.0
2023-05-02T22:00,6.2,21.5,234,239,15.8,100,0,75,100,24140.0
2023-05-02T02:00,4.7,19.1,90,44,10.1,100,0,0,100,24140.0
2023-05-02T23:00,3.7,17.4,209,232,13.3,100,0,73,100,24140.0
2023-05-02T05:00,4.6,21.4,72,48,9.7,100,0,70,100,24140.0


In [0]:
%sql
-- See the top cloud cover 
SELECT *
FROM df_merged
ORDER BY cloud_cover DESC
LIMIT 20;

time,wind_speed_10m,wind_speed_120m,wind_direction_10m,wind_direction_120m,wind_gusts_10m,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility
2023-05-02T18:00,11.5,22.6,250,253,38.2,100,0,100,100,24140.0
2023-05-01T03:00,3.2,18.8,360,357,6.8,100,0,0,100,24140.0
2023-05-04T07:00,5.1,12.5,225,252,11.5,100,62,92,100,24140.0
2023-05-02T19:00,4.0,13.5,265,286,28.1,100,0,100,80,24140.0
2023-05-02T15:00,16.5,23.7,247,246,38.2,100,0,0,100,24140.0
2023-05-02T21:00,6.8,22.1,238,240,14.8,100,0,100,100,24140.0
2023-05-02T01:00,4.0,17.2,85,37,9.4,100,0,0,100,24140.0
2023-05-02T22:00,6.2,21.5,234,239,15.8,100,0,75,100,24140.0
2023-05-02T03:00,4.8,21.4,77,45,9.7,100,0,0,100,24140.0
2023-05-02T23:00,3.7,17.4,209,232,13.3,100,0,73,100,24140.0


In [0]:
%sql
-- See if there are rows where cloud cover is 100 and there is not 100 in any other cloud cover columns
SELECT *
FROM cloud_data
WHERE cloud_cover = 100
  AND NOT (
    cloud_cover_low = 100 AND
    cloud_cover_mid = 100 AND
    cloud_cover_high = 100
  )

In [0]:
%sql
SELECT
    season, hour,
    AVG(temperature_2m) AS mean_temp,
    MIN(temperature_2m) AS min_temp,
    MAX(temperature_2m) AS max_temp
FROM df_merged
GROUP BY season, hour
ORDER BY season, hour

In [0]:
%sql
-- See key statistics for pressure features grouped by seson, hour
SELECT
    season, hour,
    AVG(surface_pressure) AS mean_surf_press,
    MIN(surface_pressure) AS min_surf_press,
    MAX(surface_pressure) AS max_surf_press,
    AVG(pressure_msl) AS mean_msl_press,
    MIN(pressure_mls) AS min_msl_press,
    MAX(pressure_msl) AS max_sml_press
FROM df_merged
GROUP BY season, hour
ORDER BY season, hour

In [0]:
%sql
-- Check for the hours where there was snow
SELECT snowfall, snow_depth
FROM df_merged
WHERE precipitation != rain + showers


In [0]:
%sql
-- Check the days where the apparent temperature 
SELECT time, apparent_temperature, temperature_2m, temperature_120m, relative_humidity_2m, dew_point_2m, vapour_pressure_deficit
FROM df_merged
WHERE apparent_temperature != temperature_2m

## Conclusions

Perpectives and features to use for clustering:

- **1. Wind and Pressure:** _wind_gusts_10m_, _wind_direction_10m_, _surface_pressure_, _pressure_msl_
- **2. Precipitation and visibility:** _cloud_cover_, _cloud_cover_high_, _visibility_, _precipitation_
- **3. Temperature and Humidity:** _temperature_2m_, _relative_humidity_2m_, _vapour_pressure_deficit_


Notes:
- Maybe add snow and snow depth in the 2 perspective (in the last 2 years there was no snow in Lisbon)
- Find more about _et0_fao_evapotranspiration_ and _dew_point_2m_ and see if they are valuable features to add
